# 1) Define all file paths

In [299]:
import os
import glob
import pandas as pd
import sqlite3

In [300]:
# Base folder where your “data” directory lives
base_dir = r"C:\Users\james\OneDrive\Documents\GitHub\solana-qrf-interval-forecasting\data"

In [301]:
# Paths to the auxiliary CSV files
csv_paths = {
    "btc_eth_price":           os.path.join(base_dir, "btc_eth_price.csv"),
    "holders":                 os.path.join(base_dir, "holders.csv"),
    "network_tx":              os.path.join(base_dir, "network_transaction_counts.csv"),
    "new_token_accounts":      os.path.join(base_dir, "new_token_accounts.csv"),
    "sol_price":               os.path.join(base_dir, "sol_price.csv"),
    "token_transfers":         os.path.join(base_dir, "token_transfers.csv"),
    "tvl":                     os.path.join(base_dir, "tvl.csv"),
    "spl_instructions":        os.path.join(base_dir, "SPL Program Instructions.csv"),
}

In [302]:
# Path to the SQLite database
db_path = os.path.join(base_dir, "solana_data_12h.db")

In [303]:
# Pattern to capture all token OHLCV CSVs under “data/tokens/”
token_csv_pattern = os.path.join(base_dir, "tokens", "*.csv")
token_files = glob.glob(token_csv_pattern)

print(f"Discovered {len(token_files)} token CSV files. Example:")
for f in token_files[:5]:
    print("   ", os.path.basename(f))
print()

Discovered 23 token CSV files. Example:
    $COLLAT.csv
    $michi.csv
    $WIF.csv
    ALCH.csv
    AVA.csv



# 2) Load and preprocess each auxiliary CSV

In [305]:
# We will store each DataFrame in a dictionary called `dfs`, keyed by a short name.
dfs = {}

In [306]:
# --- 5.1: Load btc_eth_price.csv ---
df_btceth = pd.read_csv(csv_paths["btc_eth_price"])
if "timestamp" not in df_btceth.columns:
    for col in ["time", "date", "datetime"]:
        if col in df_btceth.columns:
            df_btceth.rename(columns={col: "timestamp"}, inplace=True)
            break
df_btceth["timestamp"] = pd.to_datetime(df_btceth["timestamp"])
dfs["btc_eth_price"] = df_btceth

In [307]:
# --- 5.2: Load holders.csv (KEEP holder_count) ---
df_holders = pd.read_csv(csv_paths["holders"])
if "timestamp" not in df_holders.columns:
    for col in ["time", "date", "datetime"]:
        if col in df_holders.columns:
            df_holders.rename(columns={col: "timestamp"}, inplace=True)
            break
df_holders["timestamp"] = pd.to_datetime(df_holders["timestamp"])
# Rename token_mint → token (for consistency)
if "token_mint" in df_holders.columns:
    df_holders.rename(columns={"token_mint": "token"}, inplace=True)
# We do NOT drop holder_count—this is needed
dfs["holders"] = df_holders

In [308]:
# --- 5.3: Load network_transaction_counts.csv ---
df_net = pd.read_csv(csv_paths["network_tx"])
if "timestamp" not in df_net.columns:
    for col in ["bucket_start", "date", "datetime"]:
        if col in df_net.columns:
            df_net.rename(columns={col: "timestamp"}, inplace=True)
            break
df_net["timestamp"] = pd.to_datetime(df_net["timestamp"])
dfs["network_tx"] = df_net

In [309]:
# --- 5.4: Load new_token_accounts.csv ---
df_newacc = pd.read_csv(csv_paths["new_token_accounts"])
# Identify timestamp column as "bucket_start"
if "timestamp" not in df_newacc.columns:
    if "bucket_start" in df_newacc.columns:
        df_newacc.rename(columns={"bucket_start": "timestamp"}, inplace=True)
    else:
        for col in ["date", "datetime"]:
            if col in df_newacc.columns:
                df_newacc.rename(columns={col: "timestamp"}, inplace=True)
                break
df_newacc["timestamp"] = pd.to_datetime(df_newacc["timestamp"])
# Rename token_mint_address → token (for consistency)
if "token_mint_address" in df_newacc.columns:
    df_newacc.rename(columns={"token_mint_address": "token"}, inplace=True)
# Keep the new_token_accounts column as-is
dfs["new_token_accounts"] = df_newacc

In [310]:
# --- 2.5: Load sol_price.csv ---
df_sol = pd.read_csv(csv_paths["sol_price"])
if "timestamp" not in df_sol.columns:
    for col in ["time", "date", "datetime"]:
        if col in df_sol.columns:
            df_sol.rename(columns={col: "timestamp"}, inplace=True)
            break
df_sol["timestamp"] = pd.to_datetime(df_sol["timestamp"])
dfs["sol_price"] = df_sol

In [311]:
# --- 5.6: Load token_transfers.csv ---
df_transfers = pd.read_csv(csv_paths["token_transfers"])
# Identify timestamp column as "bucket_start"
if "timestamp" not in df_transfers.columns:
    if "bucket_start" in df_transfers.columns:
        df_transfers.rename(columns={"bucket_start": "timestamp"}, inplace=True)
    else:
        for col in ["time", "date", "datetime"]:
            if col in df_transfers.columns:
                df_transfers.rename(columns={col: "timestamp"}, inplace=True)
                break
df_transfers["timestamp"] = pd.to_datetime(df_transfers["timestamp"])
# Rename token_mint_address → token (for consistency)
if "token_mint_address" in df_transfers.columns:
    df_transfers.rename(columns={"token_mint_address": "token"}, inplace=True)
# Drop extra columns we won’t use (e.g., unique_senders, unique_receivers)
for col in ["unique_senders", "unique_receivers"]:
    if col in df_transfers.columns:
        df_transfers.drop(columns=[col], inplace=True)
# Keep transfer_count
dfs["transfers"] = df_transfers

In [312]:
# --- 2.7: Load tvl.csv ---
df_tvl = pd.read_csv(csv_paths["tvl"])
if "timestamp" not in df_tvl.columns:
    for col in ["bucket_start", "date", "datetime"]:
        if col in df_tvl.columns:
            df_tvl.rename(columns={col: "timestamp"}, inplace=True)
            break
df_tvl["timestamp"] = pd.to_datetime(df_tvl["timestamp"])
dfs["tvl"] = df_tvl

In [313]:
# --- 2.8: Load SPL Program Instructions (optional) ---
df_spl = pd.read_csv(csv_paths["spl_instructions"])
if "timestamp" not in df_spl.columns:
    for col in ["bucket_start", "date", "datetime"]:
        if col in df_spl.columns:
            df_spl.rename(columns={col: "timestamp"}, inplace=True)
            break
df_spl["timestamp"] = pd.to_datetime(df_spl["timestamp"])
dfs["spl"] = df_spl

# 3) Load and preprocess SQLite tables

In [315]:
conn = sqlite3.connect(db_path)

df_sql_tokens = pd.read_sql_query("""
    SELECT
        timestamp,
        open_usd,
        high_usd,
        low_usd,
        close_usd,
        volume_usd,
        holder_count,
        token_mint AS token,
        token_name
    FROM ohlcv_12h;
""", conn)

# Convert timestamp to datetime
df_sql_tokens["timestamp"] = pd.to_datetime(df_sql_tokens["timestamp"])

dfs["sql_tokens"] = df_sql_tokens

conn.close()

# 4) Load and preprocess each token OHLCV CSV

In [317]:
dfs_tokens = {}
for token_path in token_files:
    token_name = os.path.splitext(os.path.basename(token_path))[0]  # e.g. "$WIF"
    df_tok = pd.read_csv(token_path)

    # Drop the (empty) holder_count column if it exists in these per-token CSVs
    if "holder_count" in df_tok.columns:
        df_tok.drop(columns=["holder_count"], inplace=True)

    # Standardize timestamp column if named differently
    if "timestamp" not in df_tok.columns:
        for col in ["time", "date", "datetime"]:
            if col in df_tok.columns:
                df_tok.rename(columns={col: "timestamp"}, inplace=True)
                break
    df_tok["timestamp"] = pd.to_datetime(df_tok["timestamp"])

    # Now df_tok has no 'holder_count'; its columns are [timestamp, open_usd, high_usd, low_usd, close_usd, volume_usd, ...]
    dfs_tokens[token_name] = df_tok

# 5) Summarize all loaded DataFrames for verification

In [319]:
print("=== Summary of loaded DataFrames (auxiliary CSVs) ===\n")
for name, df in dfs.items():
    print(f"--- {name} ---")
    print("Shape      :", df.shape)
    print("Columns    :", df.columns.tolist())
    if "timestamp" in df.columns and not df["timestamp"].isnull().all():
        print("Time range :", df["timestamp"].min(), "to", df["timestamp"].max())
    else:
        print("Time range : (no timestamp column or all null)")
    print()

=== Summary of loaded DataFrames (auxiliary CSVs) ===

--- btc_eth_price ---
Shape      : (361, 9)
Columns    : ['timestamp', 'btc_open', 'btc_high', 'btc_low', 'btc_close', 'eth_open', 'eth_high', 'eth_low', 'eth_close']
Time range : 2024-12-05 12:00:00 to 2025-06-03 12:00:00

--- holders ---
Shape      : (5071, 3)
Columns    : ['timestamp', 'token', 'holder_count']
Time range : 2025-02-07 12:00:00+00:00 to 2025-06-03 11:00:00+00:00

--- network_tx ---
Shape      : (361, 2)
Columns    : ['timestamp', 'tx_count']
Time range : 2024-12-05 00:00:00+00:00 to 2025-06-03 00:00:00+00:00

--- new_token_accounts ---
Shape      : (7500, 3)
Columns    : ['token', 'timestamp', 'new_token_accounts']
Time range : 2024-12-05 00:00:00+00:00 to 2025-06-03 00:00:00+00:00

--- sol_price ---
Shape      : (361, 6)
Columns    : ['timestamp', 'open', 'high', 'low', 'close', 'volume']
Time range : 2024-12-05 12:00:00 to 2025-06-03 12:00:00

--- transfers ---
Shape      : (7502, 3)
Columns    : ['token', 'time

In [320]:
print("=== Sample per-token CSVs (first 3 tokens) ===\n")
for tok, df_tok in list(dfs_tokens.items())[:3]:
    print(f"--- Token: {tok} ---")
    print("Shape      :", df_tok.shape)
    print("Columns    :", df_tok.columns.tolist())
    print("Time range :", df_tok["timestamp"].min(), "to", df_tok["timestamp"].max())
    print()

=== Sample per-token CSVs (first 3 tokens) ===

--- Token: $COLLAT ---
Shape      : (241, 8)
Columns    : ['timestamp', 'open_usd', 'high_usd', 'low_usd', 'close_usd', 'volume_usd', 'token_mint', 'token_name']
Time range : 2025-02-03 12:00:00 to 2025-06-03 12:00:00

--- Token: $michi ---
Shape      : (301, 8)
Columns    : ['timestamp', 'open_usd', 'high_usd', 'low_usd', 'close_usd', 'volume_usd', 'token_mint', 'token_name']
Time range : 2025-01-04 12:00:00 to 2025-06-03 12:00:00

--- Token: $WIF ---
Shape      : (361, 8)
Columns    : ['timestamp', 'open_usd', 'high_usd', 'low_usd', 'close_usd', 'volume_usd', 'token_mint', 'token_name']
Time range : 2024-12-05 12:00:00 to 2025-06-03 12:00:00



# **Part 2: Master 12-hour index**

In [322]:
import numpy as np

In [323]:
# Separately, we have loaded three token-specific CSVs that need the same “per-token” treatment:
#       1. `holders_df` (columns: ["timestamp","token","holder_count"])
#       2. `transfers_df` (columns: ["timestamp","token","transfer_count"])
#       3. `new_accounts_df` (columns: ["timestamp","token","new_accounts"])

# After this step, we will have:
#  • aligned_globals[name]: each global series reindexed onto the 12h grid with forward-fill.
#  • aligned_token_tables[token][“holders”], aligned_token_tables[token][“transfers”], aligned_token_tables[token][“new_accounts”]: each token’s series reindexed onto the 12h grid (NaN where missing).
#  • aligned_tokens[token]: each token’s OHLCV from dfs_tokens reindexed onto the 12h grid (NaN where missing).


In [324]:
# 2.1 Normalize (tz-naive) and deduplicate GLOBAL tables
# --------------------------------------------------
global_names = ["btc_eth_price", "sol_price", "network_tx", "tvl"]
for name in global_names:
    df = dfs[name]
    # 1) Convert timestamp to tz-naive
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True).dt.tz_convert(None)
    # 2) Drop duplicate timestamps
    df.sort_values("timestamp", inplace=True)
    df.drop_duplicates(subset=["timestamp"], keep="last", inplace=True)
    # 3) Set timestamp index
    df.set_index("timestamp", inplace=True)
    dfs[name] = df

In [325]:
# 2.2 Normalize and deduplicate token-specific tables
#         (holders, new_token_accounts, transfers)
# --------------------------------------------------

# 2.2.a) holders_df
df_holders = dfs["holders"]
df_holders["timestamp"] = pd.to_datetime(df_holders["timestamp"], utc=True).dt.tz_convert(None)
# token column already renamed
df_holders.sort_values(["token", "timestamp"], inplace=True)
df_holders.drop_duplicates(subset=["token", "timestamp"], keep="last", inplace=True)

# 2.2.b) new_token_accounts_df
df_newacc = dfs["new_token_accounts"]
df_newacc["timestamp"] = pd.to_datetime(df_newacc["timestamp"], utc=True).dt.tz_convert(None)
# token column already renamed
df_newacc.sort_values(["token", "timestamp"], inplace=True)
df_newacc.drop_duplicates(subset=["token", "timestamp"], keep="last", inplace=True)

# 2.2.c) transfers_df
df_transfers = dfs["transfers"]
df_transfers["timestamp"] = pd.to_datetime(df_transfers["timestamp"], utc=True).dt.tz_convert(None)
# token column already renamed
df_transfers.sort_values(["token", "timestamp"], inplace=True)
df_transfers.drop_duplicates(subset=["token", "timestamp"], keep="last", inplace=True)

In [326]:
# 2.3 Normalize & deduplicate per-token OHLCV CSVs
# --------------------------------------------------
for token, df_tok in dfs_tokens.items():
    df_tok["timestamp"] = pd.to_datetime(df_tok["timestamp"], utc=True).dt.tz_convert(None)
    df_tok.sort_values("timestamp", inplace=True)
    df_tok.drop_duplicates(subset=["timestamp"], keep="last", inplace=True)
    dfs_tokens[token] = df_tok

In [327]:
# 2.4 Determine overall min/max timestamp
# --------------------------------------------------
min_times = []
max_times = []

# Global
for name in global_names:
    df = dfs[name]
    if not df.index.empty:
        min_times.append(df.index.min())
        max_times.append(df.index.max())

# Token-specific
min_times.append(df_holders["timestamp"].min())
max_times.append(df_holders["timestamp"].max())

min_times.append(df_newacc["timestamp"].min())
max_times.append(df_newacc["timestamp"].max())

min_times.append(df_transfers["timestamp"].min())
max_times.append(df_transfers["timestamp"].max())

# Per-token OHLCV
for df_tok in dfs_tokens.values():
    if not df_tok["timestamp"].isna().all():
        min_times.append(df_tok["timestamp"].min())
        max_times.append(df_tok["timestamp"].max())

overall_start = min(min_times)
overall_end   = max(max_times)
print(f"Overall data span: {overall_start} to {overall_end}")

# Build full 12h index
full_12h_index = pd.date_range(start=overall_start, end=overall_end, freq="12h")
print(f"Master 12h index length: {len(full_12h_index)}, from {full_12h_index.min()} to {full_12h_index.max()}")


Overall data span: 2024-12-05 00:00:00 to 2025-06-03 12:00:00
Master 12h index length: 362, from 2024-12-05 00:00:00 to 2025-06-03 12:00:00


In [328]:
# 2.5 Align GLOBAL tables onto full_12h_index (ffill)
# --------------------------------------------------
aligned_globals = {}
for name in global_names:
    df = dfs[name]
    df_reindexed = (
        df
        .reindex(full_12h_index)  # introduce NaN rows
        .ffill()                  # forward-fill
        .reset_index()
        .rename(columns={"index": "timestamp"})
    )
    aligned_globals[name] = df_reindexed

In [329]:
# 2.6 Align token-specific tables per token
# --------------------------------------------------

# 2.6.a) holders per token
aligned_holders = {}
for token in df_holders["token"].unique():
    df_tok = df_holders[df_holders["token"] == token].copy()
    df_tok.set_index("timestamp", inplace=True)
    df_tok = df_tok[["holder_count"]]
    df_reindexed = (
        df_tok
        .reindex(full_12h_index)
        .reset_index()
        .rename(columns={"index": "timestamp"})
    )
    df_reindexed["token"] = token
    aligned_holders[token] = df_reindexed

# 2.6.b) new_token_accounts per token
aligned_new_accounts = {}
for token in df_newacc["token"].unique():
    df_tok = df_newacc[df_newacc["token"] == token].copy()
    df_tok.set_index("timestamp", inplace=True)
    df_tok = df_tok[["new_token_accounts"]]
    df_reindexed = (
        df_tok
        .reindex(full_12h_index)
        .reset_index()
        .rename(columns={"index": "timestamp"})
    )
    df_reindexed["token"] = token
    aligned_new_accounts[token] = df_reindexed

# 2.6.c) transfers per token
aligned_transfers = {}
for token in df_transfers["token"].unique():
    df_tok = df_transfers[df_transfers["token"] == token].copy()
    df_tok.set_index("timestamp", inplace=True)
    df_tok = df_tok[["transfer_count"]]
    df_reindexed = (
        df_tok
        .reindex(full_12h_index)
        .reset_index()
        .rename(columns={"index": "timestamp"})
    )
    df_reindexed["token"] = token
    aligned_transfers[token] = df_reindexed

# 2.6.d) OHLCV CSVs per token
aligned_tokens_ohlcv = {}
for token, df_tok in dfs_tokens.items():
    df_tok.set_index("timestamp", inplace=True)
    df_reindexed = (
        df_tok
        .reindex(full_12h_index)
        .reset_index()
        .rename(columns={"index": "timestamp"})
    )
    df_reindexed["token"] = token
    aligned_tokens_ohlcv[token] = df_reindexed

In [330]:
# 2.7 Sanity Check
# --------------------------------------------------
print("\n--- Aligned global: btc_eth_price (head) ---")
print(aligned_globals["btc_eth_price"].head())

example_holder = next(iter(aligned_holders))
print(f"\n--- Aligned holders for token {example_holder} (first 3) ---")
print(aligned_holders[example_holder].head(3))

example_newacc = next(iter(aligned_new_accounts))
print(f"\n--- Aligned new_accounts for token {example_newacc} (first 3) ---")
print(aligned_new_accounts[example_newacc].head(3))

example_transfer = next(iter(aligned_transfers))
print(f"\n--- Aligned transfers for token {example_transfer} (first 3) ---")
print(aligned_transfers[example_transfer].head(3))

example_ohlcv = next(iter(aligned_tokens_ohlcv))
print(f"\n--- Aligned OHLCV for token {example_ohlcv} (first 3) ---")
print(aligned_tokens_ohlcv[example_ohlcv].head(3))


--- Aligned global: btc_eth_price (head) ---
            timestamp       btc_open       btc_high       btc_low  \
0 2024-12-05 00:00:00            NaN            NaN           NaN   
1 2024-12-05 12:00:00  103036.856923  103606.802840  96489.621485   
2 2024-12-06 00:00:00   97071.794751   98400.883900  97071.794751   
3 2024-12-06 12:00:00   98113.436509  101590.521373  97830.483957   
4 2024-12-07 00:00:00   99928.055215  100408.745127  99260.728938   

       btc_close     eth_open     eth_high      eth_low    eth_close  
0            NaN          NaN          NaN          NaN          NaN  
1   96489.621485  3934.972713  3940.376136  3806.624693  3806.624693  
2   98235.614141  3791.374120  3914.346072  3791.374120  3872.775968  
3  100648.833175  3867.363728  4067.577452  3854.354317  4031.584105  
4   99639.933180  4007.725653  4018.369292  3986.505629  3986.815519  

--- Aligned holders for token 2qEHjDLDLbuBgRYvsxhc5D6uDWAivNFZGan56P1tpump (first 3) ---
            timestamp  

#  Step 3: Merge All Aligned DataFrames into a Master Panel

In [332]:
# 3.1 Concatenate Token-Level Data
# --------------------------------------------------

# 3.1.a) OHLCV data for all tokens
# aligned_tokens_ohlcv is a dict: {token: DataFrame with columns [timestamp, open_usd, high_usd, low_usd, close_usd, volume_usd, token]}
df_ohlcv_master = pd.concat(aligned_tokens_ohlcv.values(), ignore_index=True)

# 3.1.b) Holder counts for all tokens
# aligned_holders is a dict: {token: DataFrame with columns [timestamp, holder_count, token]}
df_holders_master = pd.concat(aligned_holders.values(), ignore_index=True)

# 3.1.c) New token accounts for all tokens
# aligned_new_accounts is a dict: {token: DataFrame with columns [timestamp, new_token_accounts, token]}
df_newacc_master = pd.concat(aligned_new_accounts.values(), ignore_index=True)

# 3.1.d) Transfer counts for all tokens
# aligned_transfers is a dict: {token: DataFrame with columns [timestamp, transfer_count, token]}
df_transfers_master = pd.concat(aligned_transfers.values(), ignore_index=True)

In [333]:
# 3.2 Merge Token-Level Tables
# --------------------------------------------------

# Start from OHLCV as the base
df_tokens_combined = (
    df_ohlcv_master
    .merge(
        df_holders_master[['token', 'timestamp', 'holder_count']],
        on=['token', 'timestamp'],
        how='left'
    )
    .merge(
        df_newacc_master[['token', 'timestamp', 'new_token_accounts']],
        on=['token', 'timestamp'],
        how='left'
    )
    .merge(
        df_transfers_master[['token', 'timestamp', 'transfer_count']],
        on=['token', 'timestamp'],
        how='left'
    )
)

In [334]:
# 3.3 Merge Global Features
# --------------------------------------------------

# aligned_globals is a dict where each value is a DataFrame with columns [timestamp, <global_feature_columns>]
df_master = df_tokens_combined.copy()

# Loop over each global dataset to merge on timestamp
for name, df_glob in aligned_globals.items():
    # Rename global columns (except "timestamp") by prefixing with dataset name
    cols_to_rename = {col: f"{name}_{col}" for col in df_glob.columns if col != "timestamp"}
    df_glob_renamed = df_glob.rename(columns=cols_to_rename)
    
    # Merge into master panel
    df_master = df_master.merge(
        df_glob_renamed,
        on="timestamp",
        how="left"
    )

In [335]:
# 3.4 Reorder Columns (Optional)
# --------------------------------------------------

# Place timestamp and token first, then token-level features, then global features
token_cols = [
    "timestamp", "token",
    "open_usd", "high_usd", "low_usd", "close_usd", "volume_usd",
    "holder_count", "new_token_accounts", "transfer_count"
]

global_cols = [col for col in df_master.columns if col not in token_cols]
ordered_cols = token_cols + global_cols
df_master = df_master[ordered_cols]

In [336]:
# --------------------------------------------------
# 3.5 Inspect the Master Panel
# --------------------------------------------------

print(df_master.head())
print("\nColumns in Master Panel:")
print(df_master.columns.tolist())

            timestamp    token  open_usd  high_usd  low_usd  close_usd  \
0 2024-12-05 00:00:00  $COLLAT       NaN       NaN      NaN        NaN   
1 2024-12-05 12:00:00  $COLLAT       NaN       NaN      NaN        NaN   
2 2024-12-06 00:00:00  $COLLAT       NaN       NaN      NaN        NaN   
3 2024-12-06 12:00:00  $COLLAT       NaN       NaN      NaN        NaN   
4 2024-12-07 00:00:00  $COLLAT       NaN       NaN      NaN        NaN   

   volume_usd  holder_count  new_token_accounts  transfer_count  ...  \
0         NaN           NaN                 NaN             NaN  ...   
1         NaN           NaN                 NaN             NaN  ...   
2         NaN           NaN                 NaN             NaN  ...   
3         NaN           NaN                 NaN             NaN  ...   
4         NaN           NaN                 NaN             NaN  ...   

  btc_eth_price_eth_low btc_eth_price_eth_close  sol_price_open  \
0                   NaN                     NaN        

In [369]:
df_master

,timestamp,token,open_usd,high_usd,low_usd,close_usd,volume_usd,holder_count,new_token_accounts,transfer_count,...,btc_eth_price_eth_low,btc_eth_price_eth_close,sol_price_open,sol_price_high,sol_price_low,sol_price_close,sol_price_volume,network_tx_tx_count,tvl_tvl_usd,tvl_tvl_change_12h
0,2024-12-05 00:00:00,$COLLAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,866531.0,NaN,NaN
1,2024-12-05 12:00:00,$COLLAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3806.624693,3806.624693,242.483567,242.483567,232.376240,233.289091,2611.759253,182550905.0,NaN,NaN
2,2024-12-06 00:00:00,$COLLAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3791.374120,3872.775968,236.882044,244.364756,235.805695,236.044413,2873.570727,183188078.0,1.941217e+10,0.028605
3,2024-12-06 12:00:00,$COLLAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3854.354317,4031.584105,234.225689,240.084565,232.639141,238.073414,2852.116897,183446143.0,1.941217e+10,0.000000
4,2024-12-07 00:00:00,$COLLAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3986.505629,3986.815519,237.089680,238.549845,235.117627,238.438472,2837.946730,183675936.0,1.971885e+10,0.015798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8321,2025-06-01 12:00:00,ZEREBRO,0.029810,0.037385,0.029327,0.035637,0.390691,NaN,NaN,NaN,...,2484.639706,2538.239570,152.103430,156.893238,151.033222,156.893238,1855.822749,176590346.0,1.909351e+10,0.000000
8322,2025-06-02 00:00:00,ZEREBRO,0.036545,0.036552,0.031606,0.031826,0.403126,NaN,NaN,NaN,...,2482.897102,2483.965452,157.678225,158.278749,153.713839,153.966326,1872.837072,172739102.0,1.918176e+10,0.004622
8323,2025-06-02 12:00:00,ZEREBRO,0.031822,0.032946,0.030969,0.032946,0.384153,NaN,NaN,NaN,...,2483.016657,2581.057455,153.718959,156.300049,152.574077,156.300049,1845.678881,179729332.0,1.918176e+10,0.000000
8324,2025-06-03 00:00:00,ZEREBRO,0.032649,0.034154,0.032649,0.033673,0.404068,NaN,NaN,NaN,...,2596.048973,2614.005891,156.878012,161.951983,156.878012,161.158466,1914.444299,160288119.0,1.952288e+10,0.017783
